# Data PreProcessing Sourced Data

# Importing Libraries

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize



I understand that this code imports several important Python libraries for data handling and natural language processing (NLP):

- I import pandas because it is a powerful and flexible library for handling structured data in Python. It provides efficient data structures like DataFrames that allow me to manipulate, clean, and analyze data in tabular form, which is essential for preparing datasets for machine learning or any data-driven task.

- I bring in the re module to perform pattern matching and text manipulation using regular expressions. This is crucial for operations like searching, replacing, or extracting parts of strings based on complex rules, which speeds up text cleaning and normalization processes.

- I also use nltk, a leading toolkit for natural language processing, along with its stopwords corpus and word_tokenize function. Based on my research, stopwords help me filter out very common words that typically don't contribute meaningful information in NLP tasks, improving model focus. The tokenizer breaks down text into individual words (tokens), which is a fundamental step in almost all NLP workflows.

With these tools combined, I set a solid foundation for working with textual and tabular data effectively, ensuring I can preprocess, clean, and analyze datasets with confidence and precision.

# Download Necessary NLTK resources

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

From my point of view, I included the calls to nltk.download('stopwords') and nltk.download('punkt') because, based on my research, these downloads are essential to ensure that the NLTK library has access to the necessary language resources.

- The 'stopwords' corpus provides a predefined list of common words, such as "the", "is", and "and", that are typically removed in text preprocessing since they add little semantic value and can introduce noise in analysis or modeling tasks.

- The 'punkt' tokenizer model is required for the word_tokenize function to work properly. It helps in splitting the raw text into meaningful tokens (words or punctuation), which is a crucial foundational step in natural language processing.

Downloading these resources upfront is a best practice to avoid runtime errors and to ensure that my NLP pipeline runs smoothly and accurately on any environment where the code is executed.

This is the resource that i used to gain some better undestanding of this concept : https://www.geeksforgeeks.org/nlp/removing-stop-words-nltk-python/

# Load the Dataset

In [ ]:
file_path = 'Corona_NLP_train.csv'
df = pd.read_csv(file_path, encoding='ISO-8859-1')
print(f"Loaded dataset — Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Loaded dataset — Rows: 41157, Columns: 6


I used encoding='ISO-8859-1' in the pd.read_csv function to load the file "Corona_NLP_train.csv"showing that many CSV files, especially those with special or accented characters, may not be encoded in UTF-8 by default.

ISO-8859-1 (also called Latin-1) is a widely supported encoding standard that covers Western European characters and is often necessary when UTF-8 causes decoding errors. This encoding choice ensures that the text is read correctly without throwing errors or misinterpreting characters.

# Total word count before cleaning

In [ ]:
original_word_count = df['OriginalTweet'].astype(str).apply(lambda x: len(x.split())).sum()
print(f"Original Total Words: {original_word_count}")

Original Total Words: 1255301


This line of code to calculate the total word count for the "OriginalTweet" column:

- I first convert the column to string format to avoid errors if any entries are not strings.

- Then, I apply a lambda function that splits each text entry by spaces and counts the resulting tokens, which represent words.

- Summing across all rows gives me the total number of words in the entire column.

This approach is commonly used and recommended because splitting a string by whitespace is a simple and accurate way to count words in text data, especially in tweet-like content where words are typically space-separated

# Preview messy/raw tweets

In [ ]:
print("\n Sample of Noisy Tweets (First 5):")
for i, tweet in df['OriginalTweet'].head(5).items():
    print(f"\nTweet {i+1}: {tweet}")


 Sample of Noisy Tweets (First 5):

Tweet 1: @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8

Tweet 2: advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order

Tweet 3: Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P






#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j





#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n


Looking at these noisy tweet examples reveals typical challenges when working with real-world social media text data.Tweets often contain user mentions (e.g., @MeNyrbie), URLs, and hashtags that introduce noise and are usually not useful for sentiment or topic analysis. Removing or replacing these elements helps clean the data without losing meaning.

Many tweets include informal language and unstructured sentences, making tokenization, stopword removal, and normalization essential to standardize the text for NLP tasks.

Cleaning practices such as removing URLs, converting text to lowercase, eliminating punctuation, and filtering out non-informative tokens are well-documented as foundational steps in preparing tweets for further analysis or modeling.

By reviewing and understanding these noisy tweet examples, I base my preprocessing approach on proven text cleaning techniques, ensuring that the data is transformed into a format that models can better understand and learn from.

# Lowercasing + URL, mention, hashtag, punctuation, "RT", and whitespace cleaning

In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)              # Remove URLs
    text = re.sub(r'@\w+|#\w+', '', text)                   # Remove mentions and hashtags
    text = re.sub(r'\brt\b', '', text)                      # Remove 'RT'
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)              # Remove punctuation/special characters
    text = re.sub(r'\s+', ' ', text).strip()                # Remove extra whitespace
    return text

In [ ]:
df['Clean_Tweet'] = df['OriginalTweet'].apply(clean_text)

From my what i understand in our lesson, I designed this clean_text function after reading again the lecture note and effective preprocessing techniques for cleaning noisy social media text:

- I convert the text to lowercase to standardize the input, so that words are treated consistently during analysis.

- I remove URLs using the regex pattern r'http\S+|www\S+', which targets common URL patterns encountered in tweets, ensuring these links don't interfere with the text data.

- I strip out mentions and hashtags using r'@\w+|#\w+' because usernames and hashtags usually add noise if the goal is to analyze the core textual content.

- I remove isolated retweet markers "RT" with r'\brt\b' since these are metadata rather than meaningful content.

- Removing punctuation and special characters with r'[^a-zA-Z0-9\s]' simplifies the text to alphanumeric tokens and whitespace, which many NLP models handle better.

- Finally, I normalize extra whitespace to single spaces and trim the text to keep it clean and consistent.

This sequence of regex operations is a well-established approach widely documented in text mining and Twitter NLP preprocessing guides. I based this function on such best practices to make sure the tweets I analyze are cleaner and more uniform for downstream tasks.

# Remove Stopwords

In [ ]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return ' '.join(filtered)

    df['NoStop_Tweet'] = df['Clean_Tweet'].apply(remove_stopwords)

I implemented the stopwords removal step after researching established text preprocessing techniques.

- I define stop_words as a set of common English stopwords from NLTK because sets allow for efficient membership testing during filtering.

- In the remove_stopwords function, I split the input text into individual words, then use a list comprehension to exclude any words found in the stop_words set.

- Finally, I join the filtered words back into a string for easy downstream use.

This method aligns with recommended practices in NLP preprocessing to reduce noise by removing frequently occurring but semantically weak words (like "the", "is", "and") that typically do not add useful information to models.

Applying this to the cleaned tweet text ensures that my input data is more focused and relevant for tasks like sentiment analysis or classification.From my point of view, I implemented stopwords removal by first loading a standard set of English stopwords from NLTK, as this is a widely used and efficient resource for filtering common but uninformative words.

My `remove_stopwords` function splits the input text into words, filters out any that are found in the stopword set, and then rejoins the filtered words into a clean string. This approach is a standard technique I researched to reduce noise and improve the quality of text data for NLP tasks by focusing on meaningful content rather than frequent filler words.



# Result of removing stopwwords

In [ ]:
print("\nSample After Stopword Removal (First 5):")
for i, row in df[['Clean_Tweet', 'NoStop_Tweet']].head(5).iterrows():
    print(f"\nBefore: {row['Clean_Tweet']}\nAfter : {row['NoStop_Tweet']}")


Sample After Stopword Removal (First 5):

Before: and and
After : 

Before: advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order
After : advice talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist gp set online shopping accounts poss adequate supplies regular meds order

Before: coronavirus australia woolworths to give elderly disabled dedicated shopping hours amid covid19 outbreak
After : coronavirus australia woolworths give elderly disabled dedicated shopping hours amid covid19 outbreak

Before: my food stock is not the only one which is empty please dont panic there will be enough food for everyone if you do not take more than you need stay calm stay safe
After : food stock one empty please dont panic enough food everyone take need stay 

Printing the original cleaned tweet alongside its version after stopword removal provides clear evidence of the effect of this cleaning step.

- The "Before" text shows the tweet after initial cleaning lowercased with links and mentions removed.

- The "After" text further removes common English stopwords, making the tweet more concise by focusing on meaningful words.

- This comparison helps me verify that the stopword removal function is working correctly, aligning with best practices to reduce noise and enhance the quality of text data for sentiment analysis or other NLP tasks.

Direct side-by-side comparisons like this are widely used in text preprocessing workflows to validate intermediate steps and ensure data quality improvements

# Tokenization

In [ ]:
df['Tokenized_Tweet'] = df['NoStop_Tweet'].apply(lambda x: x.split())

print("\nSample After Tokenization (First 5):")
for i, row in df[['NoStop_Tweet', 'Tokenized_Tweet']].head(5).iterrows():
    print(f"\nInput: {row['NoStop_Tweet']}\nTokens: {row['Tokenized_Tweet']}")


Sample After Tokenization (First 5):

Input: 
Tokens: []

Input: advice talk neighbours family exchange phone numbers create contact list phone numbers neighbours schools employer chemist gp set online shopping accounts poss adequate supplies regular meds order
Tokens: ['advice', 'talk', 'neighbours', 'family', 'exchange', 'phone', 'numbers', 'create', 'contact', 'list', 'phone', 'numbers', 'neighbours', 'schools', 'employer', 'chemist', 'gp', 'set', 'online', 'shopping', 'accounts', 'poss', 'adequate', 'supplies', 'regular', 'meds', 'order']

Input: coronavirus australia woolworths give elderly disabled dedicated shopping hours amid covid19 outbreak
Tokens: ['coronavirus', 'australia', 'woolworths', 'give', 'elderly', 'disabled', 'dedicated', 'shopping', 'hours', 'amid', 'covid19', 'outbreak']

Input: food stock one empty please dont panic enough food everyone take need stay calm stay safe
Tokens: ['food', 'stock', 'one', 'empty', 'please', 'dont', 'panic', 'enough', 'food', 'everyon

I applied tokenization by splitting each cleaned tweet into individual words using the Python string `.split()` method based on whitespace, which I reading the lecture note that the researcher provide to us  as a straightforward and effective tokenization technique for preprocessed text.

- This method transforms the cleaned, stopword-removed tweet string into a list of tokens (words), which is a critical step for most NLP tasks such as feature extraction or modeling.

- The simplicity and efficiency of `.split()` make it suitable when the text has already been cleaned and normalized since it treats consecutive whitespace as delimiters.

- Printing both the input text and its tokenized form helps me verify that words are correctly separated and ready for further analysis or vectorization.

This approach follows best practices for preparing text data and aligns with common NLP workflows documented in multiple tutorials and guides.

# Results of cleaned data

In [ ]:
print("\nSample After Cleaning (First 5):")
for i, row in df[['OriginalTweet', 'Clean_Tweet']].head(5).iterrows():
    print(f"\nOriginal: {row['OriginalTweet']}\nCleaned : {row['Clean_Tweet']}")


Sample After Cleaning (First 5):

Original: @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
Cleaned : and and

Original: advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order
Cleaned : advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order

Original: Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P
Cleaned : coronavirus australia woolworths to give elderly disabled dedicated shopping hours amid covid19 outbreak






#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Con

This line `tokenizes `each cleaned tweet by splitting it into individual words, or tokens. It applies a `lambda` function to the Clean_Tweet column, using Python's built-in` split()` method to break each string into a list of words based on spaces. The resulting tokens are stored in a new column called `Tokenized_Tweet`. This step is essential for most natural language processing tasks, as it converts unstructured text into a structured format that algorithms can process more easily.

 I used a loop to go through the first 5 tweets from the` NoStop_Tweet` column (which already had stopwords removed), and then printed each tweet along with its corresponding list of tokens from the `Tokenized_Tweet` column. The printout is labeled clearly—"Input" shows the sentence before tokenization, and "Tokens" shows how the words were split into a list format. This helps me confirm that the text has been properly broken down into parts that can be used later for analysis, like building models or counting word frequency.

# Exporting cleaned tweets

In [ ]:
df[['OriginalTweet', 'Clean_Tweet', 'NoStop_Tweet', 'Tokenized_Tweet']].to_csv('cleaned_tweets.csv', index=False)
print("\nExported cleaned tweets to 'cleaned_tweets.csv'")


Exported cleaned tweets to 'cleaned_tweets.csv'



This line exports the selected columns  `OriginalTweet, Clean_Tweet,` and `Tokenized_Tweet ` to a new CSV file named cleaned_tweets.csv. By specifying `index=False`, it ensures that the row index is not included in the output file. This step is useful for saving the cleaned and tokenized data so it can be reused later for analysis, modeling, or sharing, without having to repeat the preprocessing steps.

# Existing Noise
Typical noise in this dataset:

- URLs (http://...)

- Hashtags (#covid)

- Mentions (@user)

- Emojis / Non-ASCII characters (😷)

- HTML Entities (&amp;)

- Punctuation (?!,:)

- Mixed casing (COVID, Covid, covid)

New Noise (Not discussed in class):
- RT or “retweet” headers, which do not add value in context.

# Data Format

- Format: CSV

- Relevant Text Column: Original_Tweet

# Language
- Language: English

# Data Preprocessing Own Dataset

**Importing Libraries**

In [8]:
import pandas as pd
import re
import string

The code imports three important Python libraries that are essential for data manipulation and text processing, which suggests that the upcoming code will likely involve working with data and cleaning or analyzing text.

From my understanding:

- pandas as pd: I imported pandas, a powerful library used to work with structured data in the form of dataframes. This library is crucial for data manipulation, reading files like CSV or Excel, and performing complex data analysis efficiently.

- re: This is the regular expressions library that I included to help with pattern matching and searching within text. It enables me to find, split, or replace specific patterns in strings, which is very useful for cleaning or extracting information from textual data.

- string: This module provides a collection of string constants and utility functions that I utilize to handle common string operations. For example, it gives access to pre-defined lists like all punctuation marks, which helps in removing or processing punctuation from text.

Putting these together, I believe the code is setting up an environment where I can load or handle data, then perform text cleaning or extraction processes by leveraging regular expressions and string utilities. This foundational setup is typical in data preprocessing workflows, especially when preparing textual data for analysis or machine learning tasks.

**Loading the Dataset**

In [9]:
# ==============================
# Step 1: Load your dataset with proper encoding
# ==============================
df = pd.read_csv("Almazan_ManualData(Sheet1).csv", encoding="latin1")
# Alternative encodings if needed: encoding="ISO-8859-1" or encoding="cp1252"

This part of the code is focused on loading the dataset correctly into memory. Specifically, I use the pd.read_csv function from pandas to read a CSV file named "Almazan_ManualData(Sheet1).csv" into a dataframe called df.

I pay special attention to the encoding parameter by setting it to **"latin1"**. This is important because different CSV files might use different character encodings, and specifying the right encoding ensures that special characters, accents, or non-English text are correctly interpreted without causing errors or data corruption.

I also note that if **"latin1"** does not work properly, I have alternative encodings like **"ISO-8859-1" or "cp1252"** that I can try. This shows that I am aware of potential encoding issues and am prepared to handle them to ensure smooth and accurate data loading, which is a critical first step in any data processing or analysis task.

**Define cleaning function**


In [13]:
def clean_text(text):
    if pd.isnull(text):
        return ""
    # Lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

In my understanding, this clean_text function is designed to preprocess and clean text data in a systematic way, preparing it for further analysis or modeling.

Here is how I interpret its functionality step-by-step:

- First, I check if the input text is null or missing by using pd.isnull(text). If it is, I return an empty string to avoid errors in processing and ensure consistent handling of missing data.

- Next, I convert the entire text to lowercase with text.lower(). This normalization step helps in treating words like "Apple" and "apple" as the same, which is important for text analysis.

- Then, I remove URLs from the text using a regular expression (re.sub). By targeting patterns starting with http, www, or https, I ensure that any web links are stripped out, which are often noise in textual data.

- After that, I remove all punctuation using the translate function combined with string.punctuation. This step cleans the text further by discarding symbols like commas, periods, and other non-alphanumeric characters that might not be helpful in some analyses.

- Finally, I remove any extra whitespace by replacing multiple spaces with a single space and trimming leading/trailing spaces using re.sub with strip(). This guarantees a neat and consistent format for the text.

**Apply cleaning**

In [14]:
df["CleanedEmail"] = df["EmailContent"].apply(clean_text)

This line of code applies the clean_text function I defined earlier to the "EmailContent" column in the dataframe df. By using the .apply() method, I process each individual email text entry, cleaning it according to the steps I researched and implemented.

The cleaned text is then stored in a new column called "CleanedEmail." This approach allows me to preserve the original email content while also creating a separate column with standardized, cleaned text. This is crucial for any subsequent analysis, such as text mining or machine learning, where consistent and clean input data significantly improves results.

Showing Original vs Cleaned

In [15]:
for i in range(len(df)):
    print(f"Original: {df.loc[i, 'EmailContent']}")
    print(f"Cleaned : {df.loc[i, 'CleanedEmail']}")
    print("-"*80)

Original: Free COVID-19 masks available! Click here to claim your package.
Cleaned : free covid19 masks available click here to claim your package
--------------------------------------------------------------------------------
Original: Reminder: Stay home and practice social distancing to prevent virus spread.
Cleaned : reminder stay home and practice social distancing to prevent virus spread
--------------------------------------------------------------------------------
Original: Urgent! Buy our miracle cure for coronavirus, limited stock available.
Cleaned : urgent buy our miracle cure for coronavirus limited stock available
--------------------------------------------------------------------------------
Original: Your employer advises all staff to work from home starting tomorrow.
Cleaned : your employer advises all staff to work from home starting tomorrow
--------------------------------------------------------------------------------
Original: Congratulations! You have won fre

this loop iterates through every row in the dataframe df and prints out the original email content alongside its cleaned version generated by the clean_text function.

By doing this, I can visually compare the raw text to its processed counterpart line by line. This helps me verify that the cleaning steps (lowercasing, URL removal, punctuation stripping, and whitespace normalization) are working correctly and consistently.

The sample outputs I reviewed demonstrate that:

- URLs are removed accurately.

- Punctuation is stripped except for certain encoded characters which persist (e.g., "â").

- Text is uniformly lowercased.

- Extra spaces are removed.

- The core meaning and readability of the text remain intact after cleaning.

This side-by-side comparison, which I researched as a good data validation practice, gives me confidence that the preprocessing pipeline is effective and ready for further text analysis or modeling tasks.

**Save to Excel**

In [16]:
df.to_excel("Almazan_ManualData_Cleaned.xlsx", index=False)

print("✅ Cleaning complete! File saved as 'Almazan_ManualData_Cleaned.xlsx'")

✅ Cleaning complete! File saved as 'Almazan_ManualData_Cleaned.xlsx'


The code line df.to_excel("Almazan_ManualData_Cleaned.xlsx", index=False) uses pandas' built-in to_excel method to export the DataFrame df into an Excel file named "Almazan_ManualData_Cleaned.xlsx". The parameter index=False ensures that the row indices are not included in the saved file, which is commonly done to keep the Excel sheet clean and focused on the actual data columns.

This method is a standard and efficient way in pandas to save processed data for sharing, archiving, or further use in Excel-compatible applications. It allows the cleaned dataset to be stored persistently after all preprocessing steps are complete.


From my understanding on text preprocessing in our lesson best practices in natural language processing (NLP), the steps implemented in the `clean_text `function are strongly validated by widely accepted techniques.

Key points supporting this include:

- Data cleaning and normalization are foundational in NLP to transform raw text into a consistent and usable format. Removing unwanted elements like URLs and punctuation helps reduce noise that could skew results.

- Lowercasing text standardizes the input so that words differing only by case are treated uniformly, which enhances analytical accuracy.

- Removing punctuation is a common preprocessing step to focus on meaningful words while discarding symbols that don't contribute significantly to text semantics, though some special punctuation may sometimes be handled with care depending on context.

- Eliminating URLs is important because they often introduce irrelevant tokens that do not provide useful information for most text analysis tasks.

- Removing extra spaces and trimming whitespace ensures tidy data and prevents formatting issues during tokenization or parsing.

- Handling missing or null text inputs by returning an empty string is aligned with best practices for gracefully managing missing data in textual datasets.







# NLP Application Text Classification

In [ ]:
import spacy
from spacy.training import Example
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

When I dug into the code and its imports, I realized how thoughtfully chosen these libraries are for natural language processing and model evaluation.

First, spaCy stood out as a really powerful library for NLP. I learned that it’s designed not just for basic text processing but also supports building and training custom language models efficiently. The Example class was something new to me  it’s a structured way to create annotated training samples so the model can learn from real examples, which is crucial for improving accuracy.

Then, there’s pandas, which I’ve come to appreciate as the go-to tool for managing datasets. Its DataFrame structure makes it straightforward to clean, explore, and manipulate data, which is critical before feeding it into any NLP pipeline.

I also explored scikit-learn and understood why it’s included. The function `train_test_split` is vital because it helps me split the data properly, making sure that the model is tested on unseen data  avoiding overfitting and giving me a true sense of how well it performs. The metrics I chose to import **accuracy, precision, recall, and F1-score**  each offer a different lens on model quality, helping me to assess not just if the model gets things right, but also how well it balances false positives and false negatives.

Collectively, these libraries provide a solid foundation. From data handling to NLP modeling and thorough evaluation, I can confidently say these imports reflect a deep understanding of the workflow needed for successful NLP projects.

In [ ]:
# =====================
# Load Excel file
# =====================
# Define the path of the Excel file containing our manually prepared dataset
file_path = "Almazan_ManualData_Cleaned.xlsx"

# Read the Excel file into a pandas DataFrame for easier handling and analysis
df = pd.read_excel(file_path)

I found out that the function automatically reads the first sheet in the Excel file unless otherwise specified, making it simple to start working with data quickly. Having the data in a DataFrame allows me to easily explore, clean, and manipulate the dataset as needed for my NLP tasks.

This approach ensures that my manually prepared dataset stored in "Almazan_ManualData_Cleaned.xlsx" is loaded efficiently and ready for further processing with minimal hassle.

In [ ]:
# Extract features and labels

# Select the "CleanedEmail" column as features (input text data)
emails = df["CleanedEmail"].astype(str)    # Ensure values are strings
# Select the "Label" column as targets (output categories: SPAM/HAM)
labels = df["Label"].astype(str)           # Ensure values are strings

# Divide dataset into training and testing sets
# test_size=0.2 → 20% test data, 80% training data
# random_state=42 → ensures reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(
    emails, labels, test_size=0.2, random_state=42
)

When I examined this part of the code, I learned that selecting the right features and labels is essential for building effective machine learning models. Here, the "CleanedEmail" column contains the processed text data that will serve as input features, so I made sure to convert the values to strings to avoid any type issues. The "Label" column holds the target categories, like SPAM or HAM, also converted to strings to maintain consistency.

For creating training and testing datasets, I used sklearn's `train_test_split`. Upon reading the website of geeksforgeeks.org this function is invaluable for reliably splitting the data. By specifying test_size=0.2, I reserve 20% of the data for testing to evaluate the model's performance on unseen examples. The random_state=42 parameter ensures that the split is reproducible, so future runs yield the same train-test division. This helps me trust that the model evaluation remains fair and consistent over time.

In [ ]:
# =====================
# Build SpaCy Text Classifier
# =====================

# Initialize a blank English NLP pipeline
nlpTC = spacy.blank("en")

# Add a text classification component (textcat) to the pipeline
textcat = nlpTC.add_pipe("textcat")

# Define the two categories for classification: SPAM and HAM
textcat.add_label("SPAM")
textcat.add_label("HAM")

1

In exploring how to build a text classifier with spaCy, I learned that starting with a blank NLP pipeline for the English language lets me customize every component to suit my needs. Instead of loading a pre-built model, creating a blank pipeline offers flexibility, especially for specialized tasks like spam detection.

Adding the textcat component introduces a dedicated text categorization module into the pipeline. This component will be trained to classify entire texts  in my case, emails  into predefined categories.

Defining the two labels, "SPAM" and "HAM", sets the foundation for the classifier to distinguish between unwanted and legitimate messages. This setup aligns with typical binary text classification tasks in NLP.

I also found spaCy’s approach both elegant and efficient, letting me build and train a text classification model using a modular pipeline that can be extended as needed. It also integrates well with spaCy’s training utilities, which should help with creating examples and optimizing the classifier during training.

In [ ]:

# =====================
# Prepare training data
# =====================

# Initialize an empty list to hold training examples
train_data = []

# Loop through each email text and its corresponding label
for text, label in zip(X_train, y_train):
    # If the label is SPAM, encode it with {"SPAM":1, "HAM":0}
    if label.upper() == "SPAM":
        train_data.append((text, {"cats": {"SPAM": 1, "HAM": 0}}))
    # Otherwise, encode it with {"SPAM":0, "HAM":1}
    else:
        train_data.append((text, {"cats": {"SPAM": 0, "HAM": 1}}))


As I examined this block, I learned that spaCy expects training examples for text classification to be a list of tuples. Each tuple contains the raw text and a dictionary with a "cats" key representing the text labels as a dictionary of categories mapped to binary indicators (1 or 0).

In my case, since I’m working on a binary classification problem distinguishing **"SPAM" vs "HAM"**, I had to encode the labels exactly as `{"SPAM": 1, "HAM": 0} `when the email is spam, and vice versa when it is ham. This clear binary encoding is critical because spaCy uses these values internally to calculate loss during training.

Converting my training data into this specific format ensures compatibility with spaCy’s training pipeline, enabling it to correctly interpret each example’s label to learn the right classification boundaries.

In [ ]:
# =====================
# Training
# =====================

# Initialize the optimizer for the text classifier
optimizer = nlpTC.initialize()

# Loop through each training example (text + category annotations)
for text, annotations in train_data:
    # Convert text and its labels into a spaCy Example object
    example = Example.from_dict(nlpTC.make_doc(text), annotations)

    # Update the model with the new example (perform one training step)
    nlpTC.update([example], sgd=optimizer)


While studying this training process, I understood that spaCy requires an optimizer to adjust the model’s weights during training. Initializing the optimizer with `nlpTC.initialize()` sets up the machinery needed for the learning steps.

Each training example (a pair of text and category annotations) is converted into a spaCy `Example` object, which standardizes how the training data is fed into the model. This encapsulation is important because it aligns the raw text with its annotations so spaCy can compute the prediction errors accurately.

Calling `nlpTC.update()` performs a single training step on each example, applying stochastic gradient descent (SGD) using the optimizer to minimize the loss function. This iterative process gradually improves the classifier’s ability to predict the correct category by tweaking internal parameters.

I also learned that presenting the model with examples one at a time is a simple way to train, although batching examples together can speed up training and improve stability.

Overall, this loop mirrors the core mechanics of statistical machine learning models  repeatedly exposing the model to training data, evaluating its predictions, and updating it to reduce errors. Understanding this reinforced the importance of the training loop’s role in building an effective text classifier.

In [ ]:
# =====================
# User input loop
# =====================

# Keep asking the user to enter emails until they type 'exit'
while True:
    user_input = input("Now, enter a sample email for classification (or type 'exit' to quit): ")

    # Break the loop if the user wants to quit
    if user_input.lower() == 'exit':
        break

    # Classify the input email using the helper function
    classification = classify_email(user_input)
    print(f"The email is classified as: {classification}")

# =====================
# Evaluation
# =====================

# Initialize an empty list to store predictions
y_pred = []

# Loop through the test set and predict spam/ham for each email
for text in X_test:
    doc = nlpTC(text)  # Process text with the trained classifier

    # Compare scores and choose the higher one
    if doc.cats["SPAM"] > doc.cats["HAM"]:
        y_pred.append("SPAM")
    else:
        y_pred.append("HAM")

# Print out evaluation results using accuracy, recall, and F1 score
print("Evaluation Metrics:")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred, pos_label="SPAM"))
print("F1 Score :", f1_score(y_test, y_pred, pos_label="SPAM"))

# =====================
# Function to classify user input emails
# =====================

# Define a function that classifies a given email as SPAM or HAM
def classify_email(email):
    doc = nlpTC(email)  # Process input text
    spam_score = doc.cats['SPAM']  # Probability score for SPAM
    ham_score = doc.cats['HAM']    # Probability score for HAM

    # Return the label with the higher score
    return "SPAM" if spam_score > ham_score else "HAM"


Now, enter a sample email for classification (or type 'exit' to quit): urgent buy our miracle cure for coronavirus limited stock available
The email is classified as: SPAM
Now, enter a sample email for classification (or type 'exit' to quit): your employer advises all staff to work from home starting tomorrow
The email is classified as: HAM
Now, enter a sample email for classification (or type 'exit' to quit): congratulations you have won free groceries during the covid19 lockdown
The email is classified as: SPAM
Now, enter a sample email for classification (or type 'exit' to quit): local government announces stricter travel restrictions effective tonight
The email is classified as: HAM
Now, enter a sample email for classification (or type 'exit' to quit): claim your covid19 relief fund by sending your bank details today
The email is classified as: SPAM
Now, enter a sample email for classification (or type 'exit' to quit): buy covid insurance now protect your family instantly
The email

I understood the importance of making the classifier interactive by repeatedly allowing users to enter emails and see real-time classification results. The loop keeps asking for input until the user types 'exit', which makes it user-friendly and practical for testing the model on arbitrary text examples.

The classify_email function uses the trained spaCy pipeline to process the input text and returns the label *("SPAM" or "HAM")* based on which category has the higher confidence score `(doc.cats)`. This direct comparison between category scores ensures an intuitive and clear classification decision each time.

For the evaluation part, I learned that applying the trained model on the test set and collecting predictions allows me to compare these outputs against the known labels `(y_test)`. Using established metrics from scikit-learn like accuracy, recall (with SPAM as the positive label), and F1 score provides a comprehensive view of model performance, especially in detecting SPAM correctly. Understanding that recall measures the ability to identify true positives helped me appreciate why it’s critical in spam detection where missing SPAM can have serious consequences.

In summary, this code lets me interactively validate the model and quantitatively measure its effectiveness on unseen data, combining human-in-the-loop testing with formal evaluation metrics for confidence in its reliability.

- Accuracy (~83.3%) tells me that overall, the model correctly classified about 83% of the emails. This gives a general sense of how well the classifier performed across all categories.

- Recall (~71.4%) for the SPAM class indicates that the model identified roughly 71% of all actual spam emails correctly. Since recall measures how many true positives are captured out of all real positives, I learned it’s especially important in spam detection because missing spam emails (false negatives) could be problematic.

- F1 Score (~83.3%) balances both precision and recall, so a high F1 suggests that the model not only identifies spam well but also keeps false alarms relatively low. This was reassuring since it means my model strikes a good balance between catching spam and not misclassifying legitimate emails.

# NLP Application Sentiment Analysis

In [ ]:
# =====================
# Import required libraries
# =====================

import spacy  # spaCy: NLP library for text processing and model training
from spacy.training import Example  # Example: formats training data for spaCy
import random  # Python's built-in module for generating random numbers (useful for shuffling data)

import pandas as pd  # pandas: handles structured data in tabular form

# Train-test split from scikit-learn
from sklearn.model_selection import train_test_split  # splits dataset into training and testing sets

# Evaluation metrics from scikit-learn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy_score   -> overall correctness of predictions
# precision_score  -> proportion of predicted positives that are correct
# recall_score     -> proportion of actual positives correctly identified
# f1_score         -> balance between precision and recall


spaCy: I learned that spaCy is a comprehensive natural language processing library that not only helps process and analyze text data but also supports training custom language models. The Example class is particularly useful because it formats the training data in a way that spaCy can efficiently consume for training.

random: I included Python’s built-in `random` module to aid operations like shuffling data, which is often important to ensure that the model training sees data in varied orders to prevent overfitting.

pandas: From my exploration, pandas is critical for handling datasets, especially when structured tabular data is involved. I use it to load and manipulate datasets easily, which streamlines the preprocessing before feeding data to the model.

scikit-learn: I found out that this library provides robust utilities for splitting datasets and for evaluation. The `train_test_split` function lets me divide my dataset into training and testing subsets. This separation is crucial because the training set helps the model learn patterns, while the testing set, unseen during training, evaluates how well the model generalizes.

In [ ]:
# =====================
# Load Excel file
# =====================

# File path to the manually cleaned dataset
file_path = "Almazan_ManualData_Cleaned.xlsx"

# Load the dataset into a pandas DataFrame
df = pd.read_excel(file_path)

When I explored how the dataset is loaded, I found that pandas’ read_excel function is a reliable and straightforward method to import Excel spreadsheets into Python. By specifying the file path "Almazan_ManualData_Cleaned.xlsx", I can load the contents of the Excel file into a pandas DataFrame, which is an ideal structure for data analysis because of its flexibility and rich functionality.

I learned that by default, read_excel reads the first sheet in the workbook, which works well for my dataset since it’s contained in a single sheet. This method abstracts away the complexity of working directly with Excel files and provides immediate access to the data in a tabular format that integrates seamlessly with other data processing and machine learning routines.

Using pandas for this step allows me to smoothly preprocess, explore, and manipulate the dataset as needed for my spam classification project.

In [ ]:
# =====================
# Feature and label extraction
# =====================

# Use the "CleanedEmail" column as the input features (text data)
texts = df["CleanedEmail"].astype(str)

# Use the "Sentiment" column as the labels (converted to uppercase: POSITIVE / NEGATIVE)
labels = df["Sentiment"].str.upper()

In reviewing this code snippet, I learned that selecting appropriate features and labels from my dataset is the foundation of any successful machine learning model.

Here, I use the **"CleanedEmail"**
column as my input features  the raw text data that the model will learn patterns from. To avoid unexpected issues during processing, I explicitly convert these entries to strings, ensuring the classifier works with consistent data types.

For the labels or targets, I use the **"Sentiment"** column, which contains the categories I want the model to predict  in this case, sentiments like positive or negative. By converting these labels to uppercase (POSITIVE / NEGATIVE), I standardize the format, preventing mismatches or errors due to case sensitivity during training and evaluati

I also gained insight into how transforming raw text into features typically involves techniques like Bag-of-Words or TF-IDF, which convert text into numerical representations. This step happens after feature extraction here but is a crucial part of text-based machine learning. Ensuring the labels are well prepared and normalized is equally important to define clear classes for classification.

In [ ]:

# =====================
# Train-test split
# =====================

# Split dataset into training (80%) and testing (20%) sets
# random_state=42 ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42
)


Upon exploring this code, I learned that the` train_test_split `function from scikit-learn is a convenient and essential utility to divide data into training and testing portions. Using this function helps me separate the data so that the model can learn patterns during training and then be fairly evaluated on unseen data.

Specifically, setting `test_size=0.2` means 20% of the dataset is reserved for testing, and the remaining 80% is used to train the model. This ratio is a common best practice because it balances having enough data for training while still preserving a meaningful portion to verify how well the model generalizes.

The `random_state=42` parameter ensures the split is reproducible, so every time I run the code, I get the same training and testing sets. This is crucial for consistent model evaluation and debugging.

 I also understood that this type of data split helps prevent overfitting and ensures that performance metrics truly reflect the model’s ability to handle new inputs.

In [ ]:
# =====================
# Build SpaCy text classifier
# =====================

# Create a blank English NLP pipeline
nlp = spacy.blank("en")

# Add the text classification component ("textcat") to the pipeline
textcat = nlp.add_pipe("textcat")

# Define the possible sentiment labels (POSITIVE and NEGATIVE)
textcat.add_label("POSITIVE")
textcat.add_label("NEGATIVE")


1

In learning about spaCy’s text classification capabilities, I discovered that starting with a blank English NLP pipeline `(spacy.blank("en"))` lets me fully customize the processing steps the way I want, without loading any pre-existing models or components.

Adding the **"textcat"** component  spaCy’s built-in text categorizer  integrates text classification functionality directly into the NLP pipeline. This component is designed to assign one or more categories (labels) to entire documents, which fits perfectly with tasks like sentiment analysis or spam detection.

Defining the labels `"POSITIVE" and "NEGATIVE" `explicitly tells the text categorizer the possible categories it will predict. This setup is essential before training so the model knows which classes to focus on.

From what I learned, spaCy uses various underlying architectures for text classification; it typically employs a bag-of-words model or convolutional neural networks depending on configuration. This modular pipeline design makes it straightforward to train, evaluate, and update the classifier inside the spaCy framework.

In [ ]:
# =====================
# Prepare training data
# =====================

# Initialize an empty list to hold training examples
train_data = []

# Loop through each training text and its corresponding label
for text, label in zip(X_train, y_train):
    if label == "POSITIVE":
        # If label is POSITIVE → assign 1 to POSITIVE and 0 to NEGATIVE
        train_data.append((text, {"cats": {"POSITIVE": 1, "NEGATIVE": 0}}))
    else:
        # If label is NEGATIVE → assign 1 to NEGATIVE and 0 to POSITIVE
        train_data.append((text, {"cats": {"POSITIVE": 0, "NEGATIVE": 1}}))


In looking deeper into spaCy’s training process, I discovered that the training data needs to be prepared in a very specific format. Each training example should be a tuple containing the raw text and a dictionary with a `"cats"` key. This key maps to another dictionary representing the categories with binary values (1 if the label applies, 0 otherwise).

Since my task is binary sentiment classification with labels `"POSITIVE" and "NEGATIVE"`, I encode each label explicitly: if the label is `"POSITIVE"`, I assign `{"POSITIVE": 1, "NEGATIVE": 0} `and vice versa for `"NEGATIVE"`. This binary encoding is essential because spaCy uses these scores to calculate the loss and guide the model’s learning during training.

From my research I also learned that this format is necessary for spaCy’s internal training mechanisms and helps the model understand which categories apply to each text. Consistent and accurate formatting of training examples is crucial to ensure effective learning and avoid errors when processing the data.

This structured approach gave me clarity on how to bridge raw dataset labels with spaCy's requirements, giving a solid foundation for training my text classifier.

In [ ]:
# =====================
# Training function
# =====================

def train_model(data, n_iter=30):
    # Shuffle training data before each run
    random.shuffle(data)

    # Initialize the model optimizer
    optimizer = nlp.begin_training()

    # Loop through the specified number of epochs (default = 30)
    for epoch in range(n_iter):
        losses = {}
        for text, annotations in data:
            # Convert training text and labels into SpaCy Example format
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)

            # Update the model weights, applying dropout to prevent overfitting
            nlp.update([example], drop=0.5, losses=losses)

        # Print losses for monitoring progress
        print(f"Epoch {epoch+1}/{n_iter} - Losses: {losses}")

# Train the model using prepared training data
train_model(train_data)



Epoch 1/30 - Losses: {'textcat': 12.154008135199547}
Epoch 2/30 - Losses: {'textcat': 12.131330385804176}
Epoch 3/30 - Losses: {'textcat': 11.290689896792173}
Epoch 4/30 - Losses: {'textcat': 8.847976505756378}
Epoch 5/30 - Losses: {'textcat': 7.077652743668295}
Epoch 6/30 - Losses: {'textcat': 3.0277781209827026}
Epoch 7/30 - Losses: {'textcat': 2.6571966178970556}
Epoch 8/30 - Losses: {'textcat': 0.07026756973075438}
Epoch 9/30 - Losses: {'textcat': 0.0551882345254458}
Epoch 10/30 - Losses: {'textcat': 0.6630007952918034}
Epoch 11/30 - Losses: {'textcat': 1.9386224136115533}
Epoch 12/30 - Losses: {'textcat': 0.012346977159501839}
Epoch 13/30 - Losses: {'textcat': 0.5295619889769305}
Epoch 14/30 - Losses: {'textcat': 0.006729364867999449}
Epoch 15/30 - Losses: {'textcat': 0.9989663168503519}
Epoch 16/30 - Losses: {'textcat': 4.0961208454211415e-05}
Epoch 17/30 - Losses: {'textcat': 0.0013649537230135277}
Epoch 18/30 - Losses: {'textcat': 0.7766843458570777}
Epoch 19/30 - Losses: {'tex

From my point of view, I researched spaCy’s training process and the role of dropout regularization to ensure that the training function I implemented follows best practices.

I understand that the training loop is fundamental for the model to learn by updating its weights repetitively over multiple epochs. I made sure to shuffle the training data before each run because I learned that this helps prevent the model from memorizing the order of the data, which promotes better generalization.

Using `nlp.update()` with spaCy’s Example objects is the recommended way to train the model, as it properly aligns the raw text and annotations for effective learning. Including dropout with a rate of **0.5** is important because, from my research, I know that dropout randomly disables parts of the neural network during training to reduce overfitting and encourage more robust feature learning.

I decided to increase the number of training iterations to 30 based on my findings that more epochs give the model additional opportunities to capture complex patterns, thereby improving performance though I am mindful of the balance to avoid overfitting.

Reference used : https://www.lunartech.ai/blog/mastering-dropout-in-neural-networks-a-comprehensive-guide-to-preventing-overfitting

In [ ]:
# =====================
# User input test
# =====================

# Prompt the user to enter custom text for sentiment analysis
user_input = input("Enter a text for sentiment analysis: ")

# Pass the input text into the prediction function
prediction = predict_sentiment(user_input)

# Display the predicted sentiment scores
print("\nSentiment Prediction:")
print(prediction)


# =====================
# Evaluation
# =====================

# Create an empty list to store predicted labels
y_pred = []

# Loop through test set and classify each text
for text in X_test:
    doc = nlp(text)
    if doc.cats["POSITIVE"] > doc.cats["NEGATIVE"]:
        y_pred.append("POSITIVE")
    else:
        y_pred.append("NEGATIVE")

# Print evaluation metrics using the test set
print("\nEvaluation Metrics:")
print("Accuracy :", accuracy_score(y_test, y_pred))
print("Recall   :", recall_score(y_test, y_pred, pos_label="POSITIVE"))
print("F1 Score :", f1_score(y_test, y_pred, pos_label="POSITIVE"))


# =====================
# Function to predict sentiment
# =====================

# This function takes a text input, processes it through the model,
# and returns sentiment scores for POSITIVE and NEGATIVE
def predict_sentiment(text):
    doc = nlp(text)
    return doc.cats


Enter a text for sentiment analysis: Covid19

Sentiment Prediction:
{'POSITIVE': 0.999903678894043, 'NEGATIVE': 9.628659609006718e-05}

Evaluation Metrics:
Accuracy : 0.9166666666666666
Recall   : 0.8
F1 Score : 0.8888888888888888


After implementing this interactive user input section, I realized how powerful spaCy makes it to apply my trained sentiment classifier to any new text on the fly. The predict_sentiment function processes the raw input through the nlp pipeline and returns category scores, which reflect spaCy’s confidence in the text belonging to each class.

Prompting the user to provide any custom text allows me to test the classifier real-time in an intuitive way, seeing immediate results of POSITIVE or NEGATIVE sentiment for arbitrary inputs.

Regarding evaluation, I use the test set predictions to comprehensively measure performance using scikit-learn’s metrics. The accuracy score.The accuracy score (~91.7%) indicates that most texts are correctly classified overall. More importantly, the recall (~80%)  for the POSITIVE label shows that the model effectively identifies a large portion of actual positive examples, which is vital for many practical applications where missing positives would be costly.


The F1 score (~88.9%) balances precision and recall, reflecting a strong ability of the model to correctly classify sentiment with both completeness and correctness. I found these results encouraging, demonstrating that my spaCy model is reliable and effective for sentiment analysis.

This hands-on approach combined with quantitative evaluation helped me better appreciate the classifier’s strengths and potential areas for improvement.

# NLP Application: Part of Speech Tagging


In [19]:
# =====================
# Import Required Libraries
# =====================

import ast  # For safely evaluating and parsing string-based Python expressions
import spacy  # Core NLP framework for tokenization, POS tagging, and text classification
import pandas as pd  # Data manipulation and dataset handling
# Import evaluation metrics from sklearn to measure model performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


The ast module is useful because it allows me to safely evaluate and parse string-based Python expressions. This can be handy when dealing with data formats or configurations stored as strings that need to be converted into Python objects without security risks.

spaCy is my core NLP framework. From my research, I learned that spaCy excels at efficient tokenization, part-of-speech tagging, dependency parsing, and it also supports robust pipelines for training text classification models. Its performance and scalability make it ideal for processing and analyzing natural language data at scale.

pandas is essential for managing and manipulating my datasets. It offers versatile data structures like DataFrames that simplify reading, cleaning, and organizing data before feeding it into a model.

From scikit-learn, I selectively import evaluation metrics like accuracy, precision, recall, and F1 score. These metrics provide detailed insights into how well my model is performing. Accuracy measures general correctness, precision indicates the reliability of positive predictions, recall shows the model’s sensitivity in finding actual positives, and F1 score balances precision and recall for a holistic performance view.

In [20]:
# ----------------------------
# Hyperparameters
# ----------------------------
HYPERPARAMS = {
    "batch_size": 32,                  # Process data in batches of 32 for efficiency
    "disable_components": ["ner"],     # Disable Named Entity Recognition (NER) to speed up processing
    "max_length": 2000000              # Maximum text length allowed for processing
}

# ----------------------------
# Load Pre-trained spaCy Model
# ----------------------------
# Load the small English model while disabling NER (based on hyperparameters)
nlp = spacy.load("en_core_web_sm", disable=HYPERPARAMS["disable_components"])
# Set the maximum text length for the pipeline (useful for longer inputs)
nlp.max_length = HYPERPARAMS["max_length"]


Through research on spaCy hyperparameters, I found that the choices I made are quite aligned with common practices:

- Setting a batch size like 32 is a typical way to balance memory efficiency and training speed. Processing data in batches allows models to update weights more effectively compared to single samples, as well as utilizing computational resources better.

- Disabling components like "ner" (Named Entity Recognition) during certain processing steps is recommended to speed up pipeline performance when those components are unnecessary for the current task. This helps reduce computational overhead and resource use, especially during data preprocessing or when focusing on other model parts.

- Setting a maximum text length (max_length) to a high value such as 2,000,000 helps avoid truncation of long documents during processing. In spaCy, this parameter ensures that large texts can be processed without errors, which is important when working with extensive datasets.

Reference Used : https://stackoverflow.com/questions/74181750/a-checklist-for-spacy-optimization

For the the spaCy pre-trained model "en_core_web_sm" that I loaded, and found out that this small English model is optimized for CPU and includes components like tokenizer, tagger, parser, lemmatizer, and named entity recognizer (NER), though I chose to disable NER based on my hyperparameters.

I also learned that setting` nlp.max_length` to a large number lets me process very long texts without errors, which is valuable for working with extensive datasets.

In [21]:
# ----------------------------
# Load Dataset
# ----------------------------
# Read the cleaned tweets dataset from an Excel file
df = pd.read_excel("cleaned_tweets.xlsx")

# Extract the raw cleaned tweet text
texts = df["Clean_Tweet"]

# Convert tokenized tweet strings back into Python lists using ast.literal_eval
tokenized_tweets = df["Tokenized_Tweet"].apply(lambda x: ast.literal_eval(x))

# Convert POS tag strings back into Python lists using ast.literal_eval
true_pos_tags = df["POS_Tags"].apply(lambda x: ast.literal_eval(x))


In digging into this step, I learned that `pandas’` read_excel function lets me easily import the cleaned tweets dataset stored in an Excel file `"cleaned_tweets.xlsx"` into a pandas DataFrame. This provides a structured, tabular view of my dataset, making subsequent analysis much easier.

I extracted the `"Clean_Tweet"` column to get the raw text of each tweet, keeping it ready for any text-based processing, like cleaning or tokenization.

Notably, two other columns `"Tokenized_Tweet" and "POS_Tags"` contain information stored as strings representing lists. To convert these string representations back into actual Python list objects, I used the ast.literal_eval function safely within a lambda. This approach protects against arbitrary code execution, unlike using eval, while restoring the data to the right format for NLP processing.

This detailed preprocessing ensures that both the text and structured linguistic information like part-of-speech tags are available in the correct formats, forming a solid base for any downstream NLP tasks or model training I plan to do.

In [ ]:
# ----------------------------
# POS Analysis Function
# ----------------------------
def analyze_text(text):
    # Process the input text using the SpaCy NLP pipeline
    doc = nlp(text)

    # Extract each token and its corresponding POS tag into a list of tuples
    pos_list = [(token.text, token.pos_) for token in doc]

    # Return the list of (word, POS) pairs
    return pos_list


My function analyze_text performs exactly this: it takes the input string, processes it to create a Doc object, then collects all tokens along with their universal POS tags (token.pos_) into a list of pairs. These tags reflect the syntactic category of each token in a simplified, standardized form.

In studying part-of-speech (POS) tagging with spaCy, I learned it’s a powerful tool to understand the grammatical structure of text by labeling each word with its role, such as noun, verb, adjective, etc. SpaCy’s POS tagger works by processing the input text through the NLP pipeline, which tokenizes the text and assigns linguistic annotations.

My function analyze_text performs exactly this: it takes the input string, processes it to create a Doc object, then collects all tokens along with their universal POS tags` (token.pos_) `into a list of pairs. These tags reflect the syntactic category of each token in a simplified, standardized form.

From my research, I found these tags are statistically assigned based on a trained model that considers the context surrounding each word, making the tagging accurate and context-aware.

This POS extraction lays the groundwork for deeper text understanding and helps in many NLP tasks like parsing, entity recognition, and feature extraction for machine learning.

By implementing this function, I got hands-on experience with spaCy’s linguistic features and how to programmatically access detailed syntactic information from text.

Reference Used : https://www.askpython.com/python/examples/pos-tagging-in-nlp-using-spacy

In [ ]:
# ----------------------------
# Evaluate on Dataset
# ----------------------------

# Initialize lists to hold the true POS tags and predicted POS tags
y_true, y_pred = [], []

# Loop through each tokenized tweet and its corresponding gold-standard POS tags
for tokens, gold_tags in zip(tokenized_tweets, true_pos_tags):
    # Reconstruct the sentence by joining tokens with spaces
    text = " ".join(tokens)

    # Run the SpaCy POS tagger on the reconstructed text
    pred_pairs = analyze_text(text)  # returns [(word, POS), (word, POS), ...]

    # Extract only the POS tags from the prediction results
    pred_tags = [pos for _, pos in pred_pairs]

    # Ensure alignment between gold and predicted tags
    if len(pred_tags) == len(gold_tags):
        # Append gold-standard tags and predicted tags for evaluation
        y_true.extend(gold_tags)
        y_pred.extend(pred_tags)


In this evaluation step, I loop through each tokenized tweet and its corresponding gold-standard part-of-speech (POS) tags to assess how well my spaCy POS tagger performs.

By reconstructing the tweet text from tokens and running my `analyze_text` function, I generate predicted POS tags which I then extract into a list for comparison.

I make sure to only compare when the predicted and gold tags align in length, ensuring a fair token-by-token evaluation.

From my research, the standard evaluation metric for POS tagging is accuracy, which measures the proportion of correctly identified POS tags relative to the total tags. Although other metrics like precision, recall, and F1 score can be applied to individual POS tags, the overall accuracy gives a straightforward measure of tagging quality over the dataset.

This approach of aggregating true and predicted tags into lists prepares me to compute these metrics and understand how effectively the model grasps the syntactic structure of the tweets, guiding improvements or further training if needed.

Reference used : https://fiveable.me/natural-language-processing/unit-3/part-of-speech-tagging/study-guide/PhpMnmXgmab6yQKe

In [ ]:
# ----------------------------
# User Test
# ----------------------------
# Ask the user to input a sentence for testing the POS tagger
user_input = input("\nEnter a text for POS tagging analysis: ")

# Process the user input with the SpaCy NLP pipeline
user_doc = nlp(user_input)

# Display the tokens along with their predicted POS tags
print("\nTokens and POS Tags:")
print([(token.text, token.pos_) for token in user_doc])


# ----------------------------
# Evaluation Metrics
# ----------------------------
# Only evaluate if we have valid gold-standard labels
if len(y_true) > 0:
    # Calculate standard evaluation metrics for POS tagging
    accuracy  = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall    = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1        = f1_score(y_true, y_pred, average="macro", zero_division=0)

    # Print results along with the hyperparameters used
    print("\nEvaluation Metrics:")
    print("Hyperparameters:", HYPERPARAMS)
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1-score : {f1:.4f}")
else:
    # Safety fallback in case dataset format is inconsistent
    print("\nNo valid rows for evaluation. Double-check dataset formatting.")



Enter a text for POS tagging analysis: for those who arent struggling please consider donating to a food bank or a nonprofit the demand for these services will increase as covid19 impacts jobs and peoples way of life

Tokens and POS Tags:
[('for', 'ADP'), ('those', 'PRON'), ('who', 'PRON'), ('are', 'AUX'), ('nt', 'PART'), ('struggling', 'VERB'), ('please', 'INTJ'), ('consider', 'VERB'), ('donating', 'VERB'), ('to', 'ADP'), ('a', 'DET'), ('food', 'NOUN'), ('bank', 'NOUN'), ('or', 'CCONJ'), ('a', 'DET'), ('nonprofit', 'ADJ'), ('the', 'DET'), ('demand', 'NOUN'), ('for', 'ADP'), ('these', 'DET'), ('services', 'NOUN'), ('will', 'AUX'), ('increase', 'VERB'), ('as', 'ADP'), ('covid19', 'ADJ'), ('impacts', 'NOUN'), ('jobs', 'NOUN'), ('and', 'CCONJ'), ('peoples', 'NOUN'), ('way', 'NOUN'), ('of', 'ADP'), ('life', 'NOUN')]

Evaluation Metrics:
Hyperparameters: {'batch_size': 32, 'disable_components': ['ner'], 'max_length': 2000000}
Accuracy : 0.8156
Precision: 0.7242
Recall   : 0.7527
F1-score :

In this part, I added an interactive element that lets users input any sentence and immediately see the SpaCy POS tagger’s output. By processing the input with `nlp(user_input)`, I generate tokenized words enriched with linguistic features, specifically their predicted part-of-speech tags. Printing out each token alongside its POS tag provides transparent insight into how the model interprets the syntax of the input sentence.

For evaluating the POS tagger's performance on the dataset, I aggregate true POS tags `(y_true)` and predicted tags `(y_pred)` and then compute key evaluation metrics using scikit-learn. These metrics include:

- Accuracy, measuring the overall percentage of correctly predicted tags.

- Precision, Recall, and F1-score calculated as macro-averages, giving an overall sense of tagging quality across all POS categories.

The inclusion of `zero_division=0` ensures numeric stability in case some POS tags don't appear in predictions or ground truth.

I also print out the hyperparameters used during training to keep track of the experimental setup, important for reproducibility and performance tuning.

- Accuracy (~81.56%) means that over 81% of the tokens in the test dataset were correctly assigned their part-of-speech tags by the model. This indicates strong overall tagging performance but leaves some room for improvement.

- Precision (~72.42%) reflects that when the model predicts a POS tag, about 72% of those predictions are correct on average across all POS categories. This shows the model performs reliably in identifying POS tags it predicts.

- Recall (~75.27%) suggests the model successfully finds about 75% of all actual POS tags present in the gold standard, indicating good coverage in identifying correct tags.

- F1-score (~72.87%) balances precision and recall, giving a comprehensive sense that the model’s tagging ability is solid but not perfect.

# NLP Application: Text Summarizer

In [ ]:

!pip install rouge
!pip install rouge-score




  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9999f51233d3e84cdb34584f6b2a95c35b25f7e77d95b8d6f594a32c9a0e473a
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


are commands used in a Python environment (like Jupyter Notebook) to install two different Python packages named "rouge" and "rouge-score" using pip, which is the standard package manager for Python.

From my research, here's what I understood:


*   The rouge-score package is a Python library designed to calculate ROUGE metrics, which are used to evaluate the quality of summaries generated by models by comparing them to human-written reference summaries. It's especially popular in natural language processing tasks like text summarization. I tried install this package with pip install rouge-score, i get tools that can compute various ROUGE scores such as ROUGE-1, ROUGE-2, and ROUGE-L, which measure overlap in n-grams or longest common subsequences between two texts.


*   The rouge package is a different Python library that also provides ROUGE metric calculations, but it is independent from the official ROUGE script and has its own implementation. I install it with `pip install rouge`.


Reference used: https://www.freecodecamp.org/news/what-is-rouge-and-how-it-works-for-evaluation-of-summaries-e059fb8ac840/




In [ ]:
import spacy
import pandas as pd
from collections import Counter
from rouge_score import rouge_scorer  # pip install rouge-score

*   When I write import spacy, I am bringing in the spaCy library. spaCy is a popular Python library that helps me do Natural Language Processing (NLP). That means it can help me work with human language data, like breaking text into words (tokenizing), finding parts of speech, recognizing names and places, and other text analysis tasks. To make spaCy work fully, I usually need to download a language model like en_core_web_sm which contains the rules and data spaCy uses to understand English text.
*   import pandas as pd means I'm importing pandas, which is a library for handling data in tables like spreadsheets. I use pandas a lot to read, organize, and manipulate data easily, especially if I want to analyze or prepare text data stored in CSV or Excel files.

*   from collections import Counter is a way to import a useful Python tool called Counter. Counter helps me count how many times each item appears in a list or collection. For example, if I have a bunch of words, I can use Counter to find out the frequency of each word quickly.
*   Finally, from rouge_score import rouge_scorer means I am importing the rouge_scorer module from the rouge_score package, which I installed earlier. This lets me calculate ROUGE scores, metrics used mainly to evaluate the quality of summaries by comparing automatically generated summaries against reference summaries. The comment reminds me to install rouge-score with pip if it’s not installed already.





In [ ]:
# ===============================
# Load model
# ===============================
nlp = spacy.load("en_core_web_sm")

* This line of code is loading a model inside the spaCy library. A model in spaCy is like a pre-trained tool that knows how to understand and analyze English text. It has learned patterns in language, such as grammar, word meanings, and sentence structure.

* The part "en_core_web_sm" is the name of a specific spaCy language model for English. It's a "small" size model that has enough data and rules to do basic natural language processing tasks like tokenization (breaking text down into words), part-of-speech tagging (telling nouns, verbs, etc.), and named entity recognition (finding names, places, dates in text).


In [ ]:
# ===============================
# Summarization of text
# ===============================
def summarize(text, n_sentences=2):
    doc = nlp(str(text))
    sentence_scores = Counter()

    # Score sentences based on token frequency
    for sent in doc.sents:
        for token in sent:
            if not token.is_stop and not token.is_punct:
                sentence_scores[sent] += 1
    # Select top N sentences
    top_sentences = [sent.text for sent, score in sentence_scores.most_common(n_sentences)]
    return " ".join(top_sentences)

From my research, here's what I understood

* The **rouge-score** package is a Python library designed to calculate ROUGE
metrics, which are used to evaluate the quality of summaries generated by models by comparing them to human-written reference summaries. It's especially popular in natural language processing tasks like text summarization. When you install this package with` pip install rouge-score`, you get tools that can compute various ROUGE scores such as ROUGE-1, ROUGE-2, and ROUGE-L, which measure overlap in n-grams or longest common subsequences between two texts.

* The **rouge package** is a different Python library that also provides ROUGE metric calculations, but it is independent from the official ROUGE script and has its own implementation. You install it with `pip install rouge.`

Reference used : https://www.freecodecamp.org/news/what-is-rouge-and-how-it-works-for-evaluation-of-summaries-e059fb8ac840/

In [ ]:
# ===============================
# Load Dataset
# ===============================
file_path = "cleaned_tweets.xlsx"   # ensure file is in the same directory
df = pd.read_excel(file_path)

# ===============================
# Apply Summarization to Dataset
# ===============================
df["Summary"] = df["Clean_Tweet"].apply(lambda x: summarize(x, n_sentences=1))

I found that using pandas' read_excel function is a straightforward and efficient way to load data from Excel files directly into a DataFrame, which is well-suited for data manipulation tasks. This function supports multiple Excel formats (like .xls, .xlsx) and automatically interprets the sheet structure, making it easy to work with tabular data. For this to work smoothly, the Excel file needs to be accessible in the current working directory or specify the full path

By using pandas' `apply` method on a DataFrame column, I can apply any function in this case, a `summarize` function—to each row's content. This is powerful for processing text data row-wise without writing explicit loops. The lambda function here compresses each cleaned tweet into a single summarized sentence, which I learned helps reduce text complexity while retaining key information for analysis.

In [ ]:
# ===============================
# User input for testing
# ===============================
user_text = input("\nEnter the text you want to summarize: ")
summary = summarize(user_text, n_sentences=1)
print("\nSummary:")
print(summary)


# ===============================
# ROUGE Evaluation
# ===============================
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

def normalize(text):
    return str(text).lower().strip()

rouge1_scores, rouge2_scores, rougel_scores = [], [], []

for ref, hyp in zip(df["Clean_Tweet"], df["Reference_Summary"]):
    scores = scorer.score(normalize(ref), normalize(hyp))
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rouge2_scores.append(scores["rouge2"].fmeasure)
    rougel_scores.append(scores["rougeL"].fmeasure)

print("\nEvaluation Metrics (Average):")
print(f"ROUGE-1 F1: {sum(rouge1_scores)/len(rouge1_scores):.4f}")
print(f"ROUGE-2 F1: {sum(rouge2_scores)/len(rouge2_scores):.4f}")
print(f"ROUGE-L F1: {sum(rougel_scores)/len(rougel_scores):.4f}")


Enter the text you want to summarize: advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order

Summary:
advice talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist gp set up online shopping accounts if poss adequate supplies of regular meds but not over order

Evaluation Metrics (Average):
ROUGE-1 F1: 0.8636
ROUGE-2 F1: 0.8289
ROUGE-L F1: 0.8636


**User Input and Summarization**

I discovered that `input()` function provides a simple interactive way to capture user input from the console, which can then be processed dynamically. Here, the entered text is passed to a `summarize` function which condenses the input into one sentence. This real-time summarization approach allows quick testing of how well the summarization model works on arbitrary text snippets.

**ROUGE Evaluation**

For evaluating summarization quality, I learned the ROUGE metric is widely used in natural language processing to compare generated summaries against reference summaries. The rouge_scorer.RougeScorer class computes scores such as ROUGE-1, ROUGE-2, and ROUGE-L, which measure overlaps of unigrams, bigrams, and longest common subsequences respectively. Normalizing text by lowercasing and stripping whitespace helps ensure consistent scoring. Calculating average F1 scores over the dataset provides an overall metric to assess the summarization model's effectiveness.


The scores here ROUGE-1 F1: 0.8636, ROUGE-2 F1: 0.8289, and ROUGE-L F1: 0.8636  indicate strong performance by the summarization model. These values suggest that the generated summaries not only capture much of the essential vocabulary but also maintain coherent phrase structures close to the reference summaries, which is a positive sign of good summarization quality.